In [1]:
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [3]:
edge_options = Options()
edge_options.add_argument("--disable-gpu")
edge_options.add_argument("--no-sandbox")
edge_options.add_argument("--disable-dev-shm-usage")

In [ ]:
edge_service = Service(executable_path="your_edge_driver_path")
driver = webdriver.Edge(service=edge_service, options=edge_options)

try:
    # Login
    driver.get("https://www.linkedin.com/login")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("your_email")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys("your_password")
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    print("Logged in.")

    # Open LinkedIn Post
    driver.get("post_url")
    time.sleep(5)

    # Select Most Recent Comments
    menu_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'comments-sort-order-toggle__trigger')]"))
    )
    menu_button.click()
    time.sleep(2)

    actions = ActionChains(driver)
    actions.send_keys(Keys.ARROW_DOWN).pause(0.5)
    actions.send_keys(Keys.ARROW_DOWN).pause(0.5)
    actions.send_keys(Keys.ENTER)
    actions.perform()
    print("Selected 'Most Recent Comments'.")

    # Load All Comments
    while True:
        try:
            load_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button.comments-comments-list__load-more-comments-button--cr"))
            )
            load_more_button.click()
            time.sleep(2)
        except TimeoutException:
            print("No more comments to load.")
            break

    # Save Page Source
    with open("debug_page.html", "w", encoding="utf-8") as f:
        f.write(driver.page_source)
    print("Page source saved to debug_page.html.")
finally:
    driver.quit()

Logged in.
Selected 'Most Recent Comments'.
Page source saved to debug_page.html.


In [ ]:
# Define the word to filter comments
FILTER_WORD = "example_word"

# Read the saved HTML file
with open("debug_page.html", "r", encoding="utf-8") as file:
    html_content = file.read()

# Parse the content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Extract user names, profile links, and comments
user_names = []
user_links = []
comments = []

for user in soup.find_all('span', class_='comments-comment-meta__description-title'):  
    user_names.append(user.text.strip())

for link in soup.find_all('a', class_='uBRuLksiXRXJNxzCdmDLlNvbeHoNnTVInTsbWCI tap-target overflow-hidden', href=True):
    user_links.append(link['href'])

for comment in soup.find_all('span', class_='comments-comment-item__main-content'):  
    comments.append(comment.text.strip())

df = pd.DataFrame({
    'User Name': user_names,
    'User Link': user_links,
    'Comment': comments
})


df.to_csv('linkedin_comments.csv', index=False)
print("Data saved to linkedin_comments.csv")


Data saved to linkedin_comments.csv
